# Concat multiple Databases

In [1]:
import pandas as pd
from datetime import date

## Daten laden

In [2]:
#Kommunen
bad_duerkheim = pd.read_csv('Dateien/CSV_Kommunen/bad_duerkheim_2.csv')
bad_friedrichshall= pd.read_csv('Dateien/CSV_Kommunen/bad_friedrichshall_4.csv')
bad_rappenau = pd.read_csv('Dateien/CSV_Kommunen/bad_rappenau_2.csv')
bensheim = pd.read_csv('Dateien/CSV_Kommunen/bensheim_2.csv')
frankenthal = pd.read_csv('Dateien/CSV_Kommunen/frankenthal_2.csv')
hassloch = pd.read_csv('Dateien/CSV_Kommunen/hassloch_2.csv')
heppenheim = pd.read_csv('Dateien/CSV_Kommunen/heppenheim_2.csv')
hockenheim = pd.read_csv('Dateien/CSV_Kommunen/hockenheim_2.csv')
homburg = pd.read_csv("Dateien/CSV_Kommunen/homburg_2.csv")
lampertheim = pd.read_csv("Dateien/CSV_Kommunen/lampertheim_2.csv")
landau = pd.read_csv("Dateien/CSV_Kommunen/landau_2.csv")
leimen = pd.read_csv("Dateien/CSV_Kommunen/leimen_2.csv")
pirmasens = pd.read_csv('Dateien/CSV_Kommunen/pirmasens_2.csv')
schwetzingen = pd.read_csv('Dateien/CSV_Kommunen/schwetzingen_2.csv')
schifferstadt = pd.read_csv('Dateien/CSV_Kommunen/schifferstadt_4.csv')
speyer = pd.read_csv('Dateien/CSV_Kommunen/speyer_5.csv')
viernheim = pd.read_csv('Dateien/CSV_Kommunen/viernheim_2.csv')
weinheim = pd.read_csv('Dateien/CSV_Kommunen/weinheim_2.csv')
wiesloch = pd.read_csv('Dateien/CSV_Kommunen/wiesloch_2.csv')
woerth_am_rhein = pd.read_csv('Dateien/CSV_Kommunen/woerth_am_rhein_2.csv')

#Kreise
kreis_bad_duerkheim = pd.read_csv('Dateien/CSV_Kreise/kreis_bad_duerkheim_2.csv')
kreis_bergstrasse = pd.read_csv('Dateien/CSV_Kreise/kreis_bergstrasse_2.csv')
kreis_germersheim = pd.read_csv('Dateien/CSV_Kreise/kreis_germersheim_2.csv')
kreis_heilbronn = pd.read_csv('Dateien/CSV_Kreise/kreis_heilbronn.csv')
rhein_neckar_kreis = pd.read_csv('Dateien/CSV_Kreise/rhein_necker_kreis.csv')
rhein_pfalz_kreis = pd.read_csv('Dateien/CSV_Kreise/rhein_pfalz_kreis_4.csv')
saarpfalz_kreis = pd.read_csv('Dateien/CSV_Kreise/saarpfalz_kreis.csv')

# Einzelne Tabellen zusammenführen

In [3]:
kommunen = [#Kommunen
            bad_duerkheim,
            bad_friedrichshall,
            bad_rappenau,
            bensheim,
            frankenthal,
            hassloch,
            heppenheim,
            hockenheim,
            homburg,
            lampertheim,
            landau,
            leimen,
            pirmasens,
            schwetzingen,
            schifferstadt,
            speyer,
            viernheim,
            weinheim,
            wiesloch,
            woerth_am_rhein,
            
            #Kreise
            kreis_bad_duerkheim,
            kreis_bergstrasse,
            kreis_germersheim,
            kreis_heilbronn,
            rhein_neckar_kreis,
            rhein_pfalz_kreis,
            saarpfalz_kreis,
           ]

# Datensätze zusammenführen
df = pd.concat(kommunen, ignore_index=True)

# Überschriften herausfiltern
df = df[df['content'] != 'content']

## Datentypen festlegen

In [4]:
df.drop('files', axis=1, inplace=True)
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%d")
df['id'] = pd.to_numeric(df['id'], errors='coerce',downcast='signed')
df['sitzung_nr'] = pd.to_numeric(df['id'], errors='coerce',downcast='signed')
df['top_nr'] = df['top_nr'].astype('str')
df['vorlage_nr'] = df['vorlage_nr'].astype('str')
df['kommunale_ebene'] = df['kommunale_ebene'].astype('category')
df['kommune'] = df['kommune'].astype('category')

## Daten zwischenspeichern

In [199]:
df.to_parquet(f"{date.today()}_mittelstaedte_rhein_neckar.parquet")

# Get PDF Content

In [5]:
import fitz
import requests
import os
import numpy as np
from tqdm import tqdm

In [6]:
def extract_text(pdf_file):
    try:
        pdf_text = ''
        with fitz.open(pdf_file) as pdf:
            for page in pdf:
                pdf_text += page.get_text()
    except:
        pdf_text = None

    return pdf_text

In [7]:
def download_file(url, path):
    try:
        file = requests.get(url)
        with open(path , 'wb') as f:
            f.write(file.content)
    except:
        pass

In [209]:
df_copy = df.copy()

## Durch gesamte Datenbank iterieren und PDF-Inhalte extrahieren

In [8]:
for i in tqdm(df.index, total=df.shape[0]):
    
    # Nur weitermachen, wenn Zelle 'content' leer ist
    if pd.isnull(df['content'][i]) or df['content'][i] == '':
        path = df['rel_path_to_file'][i] # Dateipfad
        
        # Nur weitermachen, wenn ein Dateipfad angegeben ist
        if pd.notnull(path):
            
            # Wenn Datei nicht bereits heruntergeladen wurde, dann Datei herunterladen
            if os.path.isfile(path) == False:
                url = df['file_urls'][i]
                download_file(url, path)
            
            # Text extrahieren
            df.at[i, 'content'] = extract_text(path)

# Daten zwischenspeichern
df.to_parquet("2023-03-24_gesamt_rhein_neckar.parquet")

100%|███████████████████████████████████████████████████████████████████████| 70083/70083 [53:14<00:00, 21.94it/s]


In [3]:
df = pd.read_parquet("2023-03-24_gesamt_rhein_neckar.parquet")

In [4]:
df_no_content =df[(pd.isnull(df['content']))|(df['content'] == '')]

In [11]:
df_no_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9157 entries, 2 to 70033
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   content           8403 non-null   object        
 1   date              8961 non-null   datetime64[ns]
 2   doc_name          8871 non-null   object        
 3   doc_typ           8571 non-null   object        
 4   file_urls         7615 non-null   object        
 5   gremium           9157 non-null   object        
 6   id                8673 non-null   float64       
 7   kommunale_ebene   9157 non-null   category      
 8   kommune           9157 non-null   category      
 9   pdf_name          8871 non-null   object        
 10  rel_path_to_file  8871 non-null   object        
 11  sitzung_nr        8673 non-null   float64       
 12  status            2672 non-null   object        
 13  top_name          8188 non-null   object        
 14  top_nr            9157 

## OCR

In [5]:
import tempfile
from pdf2image import convert_from_path
import pytesseract
import os
import time
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from tqdm import tqdm

In [6]:
myconfig = r'-l deu'
poppler_path = r'C:\Program Files\poppler-0.68.0\bin'
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [7]:
def ocr(path, poppler_path):
    if pd.notnull(path):
        try:
            doc_img = convert_from_path(path, poppler_path=poppler_path,)
            pdf_text = ''
            for page in doc_img:
                pdf_text += pytesseract.image_to_string(page, config=myconfig, lang='deu')
            return pdf_text
        except:
            return 'OCR_FAILED'

In [8]:
df['ocr'] = (pd.isnull(df['content']))|(df['content'] == '')

In [9]:
for i in tqdm(df_no_content.index, total=df_no_content.shape[0]):
    file_path = df['rel_path_to_file'][i]
    pdf_text = ocr(file_path, poppler_path)
    df.at[i, 'content'] = pdf_text
df.to_parquet("2023-03-25_gesamt_ocr_rhein_neckar.parquet")

 11%|███████▌                                                            | 1022/9157 [3:55:52<13:08:12,  5.81s/it]C:\Users\johan\OneDrive_HSRM\Studium\Masterthesis\py_env_masterthesis\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (132283200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 12%|████████▍                                                           | 1135/9157 [4:18:15<23:11:37, 10.41s/it]C:\Users\johan\OneDrive_HSRM\Studium\Masterthesis\py_env_masterthesis\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (119064826 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 43%|█████████████████████████████                                      | 3973/9157 [11:53:14<12:56:02,  8.98s/it]C:\Users\johan\OneDrive_HSRM\Studium\Masterthesis\py_env_masterthesis\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (10614

In [15]:
df_no_content_ocr = df[df['content'] == 'OCR_FAILED']

In [16]:
df_no_content_ocr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 571 entries, 668 to 69956
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   content           571 non-null    object        
 1   date              476 non-null    datetime64[ns]
 2   doc_name          571 non-null    object        
 3   doc_typ           571 non-null    object        
 4   file_urls         495 non-null    object        
 5   gremium           571 non-null    object        
 6   id                474 non-null    float64       
 7   kommunale_ebene   571 non-null    category      
 8   kommune           571 non-null    category      
 9   pdf_name          571 non-null    object        
 10  rel_path_to_file  571 non-null    object        
 11  sitzung_nr        474 non-null    float64       
 12  status            280 non-null    object        
 13  top_name          467 non-null    object        
 14  top_nr            571 